# Sequential Chats and Customer Onboarding

### Setup

In [2]:
from utils import get_openai_api_key

from autogen import ConversableAgent
from autogen import initiate_chats

/home/muriloms/Documents/rep/ai-agent-projects/autogen/venv/lib/python3.11/site-packages/flaml/__init__.py:20: UserWarning: flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.
  warnings.warn("flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.")


In [3]:
OPEN_AI_KEY = get_openai_api_key()
llm_config={
    "model":"gpt-4o-mini"
}

### Cheating the needed agents

In [4]:
onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding Personal Information Agent",
    system_message=''' 
    You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask other information.
    Retunr 'TERMINATE' when you have gathered all the information.
    ''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode='NEVER'
)

[autogen.oai.client: 01-13 16:56:25] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [5]:
onboarding_topic_preference_agent = ConversableAgent(
    name="Onborading Topic Preference Agent",
    system_message=''' 
    You are a helpful customer onboarding agent.
    You are here to help new customer's get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Retunr 'TERMINATE' when you have gathered all the information.
    ''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode='NEVER'
)

[autogen.oai.client: 01-13 16:56:25] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [6]:
customer_engagement_agent = ConversableAgent(
    name="Customer Engagement Agent",
    system_message='''You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

[autogen.oai.client: 01-13 16:56:25] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [7]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

### Creating tasks

In [12]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": 
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": 
                "Great! Could you tell me what topics you are "
                "interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

### Start the onboarding process

In [13]:
chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Personal Information Agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product.Could you tell me your name and location?

--------------------------------------------------------------------------------

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
customer_proxy_agent (to Onboarding Personal Information Agent):



--------------------------------------------------------------------------------
Onboarding Personal Information Agent (to customer_proxy_agent):

It seems like I didn't receive your response. Could you please provide your name and location?

--------------------------------------------------------------------------------
customer_proxy_agent (to Onboarding Personal Information Agent):

murilo, brazil

--------------------------

In [14]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

```json
{
  "name": "murilo",
  "location": "brazil"
}
```


Murilo, who is located in Brazil, is asked about his interests in reading topics.


Murilo is encouraged to explore fun reading material related to Brazilian culture, nature, and humor. The suggestions include interesting facts about Brazil, a joke, and a short story about a girl discovering a hidden waterfall in the Amazon, emphasizing the importance of friendship and nature conservation.




In [15]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0, 'gpt-4o-mini-2024-07-18': {'cost': 0, 'prompt_tokens': 548, 'completion_tokens': 124, 'total_tokens': 672}}, 'usage_excluding_cached_inference': {'total_cost': 0, 'gpt-4o-mini-2024-07-18': {'cost': 0, 'prompt_tokens': 268, 'completion_tokens': 63, 'total_tokens': 331}}}


{'usage_including_cached_inference': {'total_cost': 0, 'gpt-4o-mini-2024-07-18': {'cost': 0, 'prompt_tokens': 354, 'completion_tokens': 63, 'total_tokens': 417}}, 'usage_excluding_cached_inference': {'total_cost': 0, 'gpt-4o-mini-2024-07-18': {'cost': 0, 'prompt_tokens': 354, 'completion_tokens': 63, 'total_tokens': 417}}}


{'usage_including_cached_inference': {'total_cost': 0, 'gpt-4o-mini-2024-07-18': {'cost': 0, 'prompt_tokens': 1431, 'completion_tokens': 948, 'total_tokens': 2379}}, 'usage_excluding_cached_inference': {'total_cost': 0, 'gpt-4o-mini-2024-07-18': {'cost': 0, 'prompt_tokens': 1431, 'completion_tokens': 948, 'total_tokens': 2379}}}


